In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,231 kB]
Fetched 2,482 kB in 4s (685 kB/s)
Reading package lists... Done


Install Java, as Spark depends on it

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# get spark
VERSION='3.5.0'
!wget https://dlcdn.apache.org/spark/spark-$VERSION/spark-$VERSION-bin-hadoop3.tgz

--2023-11-04 17:15:10--  https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz.2’

spark-3.5.0-bin-had 100%[===================>] 381.85M   198MB/s    in 1.9s    

2023-11-04 17:15:12 (198 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz.2’ saved [400395283/400395283]



Download Spark and decompress it

In [ ]:
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark

Set up environment variables for Java and Spark

In [ ]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3"

Initialize Spark using findspark

In [ ]:
# add pyspark to sys.path using findspark
import findspark
findspark.init()

Create a Spark session

In [ ]:
# get a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Download the dataset directly to the environment and read it using Spark

In [ ]:
!wget https://storage.googleapis.com/bdt-demand-forecast/sales-data.csv -O gcs_sales_data.csv

--2023-11-04 17:15:25--  https://storage.googleapis.com/bdt-demand-forecast/sales-data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.114.207, 172.217.212.207, 108.177.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.114.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17333449 (17M) [text/csv]
Saving to: ‘gcs_sales_data.csv’

gcs_sales_data.csv  100%[===================>]  16.53M  17.3MB/s    in 1.0s    

2023-11-04 17:15:27 (17.3 MB/s) - ‘gcs_sales_data.csv’ saved [17333449/17333449]



Read the data from the downloaded CSV file

In [ ]:
df = spark.read.csv('gcs_sales_data.csv', header=True, inferSchema=True)

In [ ]:
df.show()

+----------+-----+----+-----+
|      date|store|item|sales|
+----------+-----+----+-----+
|2013-01-01|    1|   1|   13|
|2013-01-02|    1|   1|   11|
|2013-01-03|    1|   1|   14|
|2013-01-04|    1|   1|   13|
|2013-01-05|    1|   1|   10|
|2013-01-06|    1|   1|   12|
|2013-01-07|    1|   1|   10|
|2013-01-08|    1|   1|    9|
|2013-01-09|    1|   1|   12|
|2013-01-10|    1|   1|    9|
|2013-01-11|    1|   1|    9|
|2013-01-12|    1|   1|    7|
|2013-01-13|    1|   1|   10|
|2013-01-14|    1|   1|   12|
|2013-01-15|    1|   1|    5|
|2013-01-16|    1|   1|    7|
|2013-01-17|    1|   1|   16|
|2013-01-18|    1|   1|    7|
|2013-01-19|    1|   1|   18|
|2013-01-20|    1|   1|   15|
+----------+-----+----+-----+
only showing top 20 rows



In [ ]:
# Required Libraries
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

Prepare Data by Partitioning

In [ ]:
# Prepare Data: Repartition based on 'store' and 'item' for parallel processing
store_item_history = df.repartition(col("store"), col("item"))

# Convert the 'date' column to string format
store_item_history = store_item_history.withColumn("date", col("date").cast("string"))

In [ ]:
store_item_history.show(3)

+----------+-----+----+-----+
|      date|store|item|sales|
+----------+-----+----+-----+
|2013-01-01|    3|  22|   50|
|2013-01-02|    3|  22|   53|
|2013-01-03|    3|  22|   51|
+----------+-----+----+-----+
only showing top 3 rows



Model Fit & Forecast for Each Store-Item Combination

In [ ]:
from prophet import Prophet
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType, StringType, StructField, StructType, DoubleType
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pyspark.sql import SparkSession

# Adjust the output schema to include a 'forecast' field
schema = StructType([
    StructField("date", StringType()),  # 'date' to a StringType or else fails
    StructField("store", IntegerType()),
    StructField("item", IntegerType()),
    StructField("sales", IntegerType()),  # Historical sales
    StructField("forecast", DoubleType())  # Forecasted sales
])

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def prophet_fit_forecast_function(pdf):
    # Fit the model
    model = Prophet(daily_seasonality=True)
    model.fit(pdf.rename(columns={'date': 'ds', 'sales': 'y'}))

    # Create a future dataframe for forecasting
    future = model.make_future_dataframe(periods=0)  # Can always adjust periods if forecasting future beyond the historical data

    # Forecast
    forecast = model.predict(future)

    # Assign the forecasted 'yhat' to a new 'forecast' column
    pdf['forecast'] = forecast['yhat'].values

    # Ensure we return the DataFrame with the structure defined in the schema
    pdf['date'] = pdf['date'].astype(str)  # Convert date back to string if necessary

    return pdf[['date', 'store', 'item', 'sales', 'forecast']]  # Return the structured DataFrame

# Apply the UDF to forecast
store_item_forecasts = store_item_history.groupBy('store', 'item').apply(prophet_fit_forecast_function)

Persist Forecasts for Evaluation

In [ ]:
path_to_save = "forecasts.csv"
store_item_forecasts.write.mode("overwrite").csv(path_to_save)

Model Evaluation

In [ ]:
def evaluate_forecast(evaluation_pd):
    mae = mean_absolute_error(evaluation_pd['sales'], evaluation_pd['forecast'])
    mse = mean_squared_error(evaluation_pd['sales'], evaluation_pd['forecast'])
    rmse = sqrt(mse)

    results = {'mae':[mae], 'mse':[mse], 'rmse':[rmse]}
    return pd.DataFrame.from_dict(results)

evaluation_results = store_item_forecasts.groupBy('store', 'item').applyInPandas(evaluate_forecast, schema="mae float, mse float, rmse float")

Print Evaluation Results

In [ ]:
evaluation_results.show()

+---------+---------+---------+
|      mae|      mse|     rmse|
+---------+---------+---------+
|3.5115125|19.698908| 4.438345|
|6.1717906| 60.78164| 7.796258|
| 4.682532|35.544044|5.9618826|
|3.6469831|20.626503| 4.541641|
|6.1389756| 59.68847|7.7258315|
| 7.155143|  80.2021| 8.955563|
|5.6625113|50.739956|7.1231985|
| 6.642689|  69.9807| 8.365447|
|3.9194186|24.434675|4.9431443|
|5.1415143|41.410084|6.4350667|
|5.2595453|44.609444|  6.67903|
| 7.498411|88.230385| 9.393104|
|  4.32822|29.853817| 5.463865|
|7.0863786|78.689514| 8.870711|
|5.4263525|46.505756| 6.819513|
|3.6331272| 20.56307| 4.534652|
|5.2866144|  43.9465|6.6292157|
|6.6468644| 69.84013| 8.357041|
| 6.970256| 77.73334| 8.816651|
|4.3034377|29.081266| 5.392705|
+---------+---------+---------+
only showing top 20 rows



2) Number of partitions in the store_item_history dataframe

In [ ]:
print(f"Number of partitions: {store_item_history.rdd.getNumPartitions()}")

Number of partitions: 2


3) Parallelise the workload and Demonstrate Parallisation

In [ ]:
from pyspark.sql import SparkSession
import time

# Function to time the operation
def time_operation(spark_session, operation):
    start_time = time.time()
    operation(spark_session)
    end_time = time.time()
    return end_time - start_time

# Create a Spark session with a single core
spark_single_core = SparkSession.builder.master("local[1]").appName("SingleCore").getOrCreate()

# Create another Spark session using all available cores
spark_multi_core = SparkSession.builder.master("local[*]").appName("MultiCore").getOrCreate()

# Define your DataFrame here (for the sake of an example, we're creating a simple DataFrame with dummy data)
data = [("Store1", "Item1", 1), ("Store2", "Item2", 2), ("Store1", "Item1", 3)]
columns = ["store", "item", "sales"]
df = spark_multi_core.createDataFrame(data, columns)

# Define the operation you want to time
def grouped_count(spark_session):
    df.groupBy('store', 'item').count().collect()

# Time the operation without parallelization (using a single core)
non_parallel_duration = time_operation(spark_single_core, grouped_count)

# Time the operation with parallelization (using all available cores)
parallel_duration = time_operation(spark_multi_core, grouped_count)

# Print the durations
print(f"Duration without parallelization (single core): {non_parallel_duration} seconds")
print(f"Duration with parallelization (all cores): {parallel_duration} seconds")

# Stop the Spark sessions
spark_single_core.stop()
spark_multi_core.stop()

Duration without parallelization (single core): 1.471919298171997 seconds
Duration with parallelization (all cores): 0.4441835880279541 seconds
